In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

In [2]:
path_to_nwp = './nwpdata/20220101.zarr'
path_to_pv = './pvdata/pv/italy/PV_PVOutput.org_Italy_PVOutput_Italy_systems_metadata.csv'

In [7]:
nwp_data = xr.open_dataset(path_to_nwp, engine='zarr', chunks='auto')
pv_data = pd.read_csv(path_to_pv)

In [8]:
nwp_data

<xarray.Dataset>
Dimensions:    (time: 18, latitude: 121, longitude: 113)
Coordinates:
  * latitude   (latitude) float32 58.0 57.75 57.5 57.25 ... 28.5 28.25 28.0
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 27.25 27.5 27.75 28.0
  * time       (time) datetime64[ns] 2022-01-01T05:00:00 ... 2022-01-01T22:00:00
Data variables: (12/60)
    alnid      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    alnip      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    aluvd      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    aluvp      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    bld        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    blh        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    ...         ...
    u100       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    u10n       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    uvb        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    v100       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    wind       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-11-26 09:57:56 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [9]:
pv_data

,Unnamed: 0,name,system_DC_capacity_W,address,num_panels,panel_capacity_W_each,panel_brand,num_inverters,inverter_capacity_W,inverter_brand,...,shade,install_date,latitude,longitude,status_interval_minutes,secondary_num_panels,secondary_panel_capacity_W_each,secondary_orientation,secondary_array_tilt_degrees,system_id
0,0,Pergola Fotovoltaica,9165,NaN,39,235,LG,1,10000,Power One PVI 10.0 OUTD-S,...,No,2011-02-28,36.715286,15.091528,5,0,0,NaN,NaN,24000
1,1,Saiazza,14000,96018.0,70,200,Deluxe,1,12500,Aurora,...,No,NaN,36.718600,15.090700,5,0,0,NaN,NaN,48316
2,2,Tetto Fotovoltaico Saiazza,14000,96018.0,70,200,Deluxe,1,12500,Aurora,...,No,NaN,36.718600,15.090700,5,0,0,NaN,NaN,41556
3,3,Scalecci,13230,96017.0,63,210,Centrosolar,1,12500,Power One PVI 12.5 OUTD FS,...,No,2013-02-25,36.734759,15.098036,5,0,0,NaN,NaN,24001
4,4,Marzamemi-FV,2990,96017.0,13,230,LG 230W,1,3000,Power One Aurora PVI 3.0,...,Medium,2011-04-28,36.747195,15.108724,5,0,0,NaN,NaN,24760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,1320,Test 11,50,39031.0,1,50,NaN,1,50,Energy Monitor,...,No,NaN,46.795800,11.936400,5,0,0,NaN,NaN,60986
1321,1321,OptisolarSos,1000,33020.0,4,250,NaN,1,4000,OptiSolar,...,No,2014-11-04,46.518650,12.835684,5,0,0,NaN,NaN,33148
1322,1322,sonneundsee,5280,NaN,22,240,Schueco,1,5500,Kostal,...,No,2012-09-01,46.687427,13.975522,10,0,0,NaN,NaN,20925
1323,1323,DV,382200,39049.0,1274,300,Sunrise,26,5500,Solutronic,...,No,2010-12-31,46.896300,11.447600,10,0,0,NaN,NaN,79743


In [11]:
pv_data[['latitude', 'longitude']].min()

latitude     36.715286
longitude     6.738385
dtype: float64

In [12]:
pv_data[['latitude', 'longitude']].max()

latitude     46.896300
longitude    18.422746
dtype: float64

In [14]:
nwp_data

<xarray.Dataset>
Dimensions:    (time: 18, latitude: 121, longitude: 113)
Coordinates:
  * latitude   (latitude) float32 58.0 57.75 57.5 57.25 ... 28.5 28.25 28.0
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 27.25 27.5 27.75 28.0
  * time       (time) datetime64[ns] 2022-01-01T05:00:00 ... 2022-01-01T22:00:00
Data variables: (12/60)
    alnid      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    alnip      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    aluvd      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    aluvp      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    bld        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    blh        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    ...         ...
    u100       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    u10n       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    uvb        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    v100       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    wind       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-11-26 09:57:56 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [17]:
pv_data['nearest_latitude'] = round(pv_data['latitude'] * 4) /4
pv_data['nearest_longitude'] = round(pv_data['longitude'] * 4) /4

In [19]:
def map_val(value: float) -> float:
    return round(value * 4) / 4

In [20]:
pv_data

,Unnamed: 0,name,system_DC_capacity_W,address,num_panels,panel_capacity_W_each,panel_brand,num_inverters,inverter_capacity_W,inverter_brand,...,latitude,longitude,status_interval_minutes,secondary_num_panels,secondary_panel_capacity_W_each,secondary_orientation,secondary_array_tilt_degrees,system_id,nearest_latitude,nearest_longitude
0,0,Pergola Fotovoltaica,9165,NaN,39,235,LG,1,10000,Power One PVI 10.0 OUTD-S,...,36.715286,15.091528,5,0,0,NaN,NaN,24000,36.75,15.00
1,1,Saiazza,14000,96018.0,70,200,Deluxe,1,12500,Aurora,...,36.718600,15.090700,5,0,0,NaN,NaN,48316,36.75,15.00
2,2,Tetto Fotovoltaico Saiazza,14000,96018.0,70,200,Deluxe,1,12500,Aurora,...,36.718600,15.090700,5,0,0,NaN,NaN,41556,36.75,15.00
3,3,Scalecci,13230,96017.0,63,210,Centrosolar,1,12500,Power One PVI 12.5 OUTD FS,...,36.734759,15.098036,5,0,0,NaN,NaN,24001,36.75,15.00
4,4,Marzamemi-FV,2990,96017.0,13,230,LG 230W,1,3000,Power One Aurora PVI 3.0,...,36.747195,15.108724,5,0,0,NaN,NaN,24760,36.75,15.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,1320,Test 11,50,39031.0,1,50,NaN,1,50,Energy Monitor,...,46.795800,11.936400,5,0,0,NaN,NaN,60986,46.75,12.00
1321,1321,OptisolarSos,1000,33020.0,4,250,NaN,1,4000,OptiSolar,...,46.518650,12.835684,5,0,0,NaN,NaN,33148,46.50,12.75
1322,1322,sonneundsee,5280,NaN,22,240,Schueco,1,5500,Kostal,...,46.687427,13.975522,10,0,0,NaN,NaN,20925,46.75,14.00
1323,1323,DV,382200,39049.0,1274,300,Sunrise,26,5500,Solutronic,...,46.896300,11.447600,10,0,0,NaN,NaN,79743,47.00,11.50


In [23]:
pv_timeseries = pd.read_csv('pvdata/pv/italy/pv_time_series.csv')
pv_timeseries

,system_id,timestamp,instantaneous_power_W,temperature_C,voltage
0,10441,2015-01-01 08:00:00,108.000000,2.933333,231.000000
1,10441,2015-01-01 08:15:00,573.333333,3.233333,229.333333
2,10441,2015-01-01 08:30:00,780.000000,6.000000,230.333333
3,10441,2015-01-01 08:45:00,1002.666667,3.666667,232.000000
4,10441,2015-01-01 09:00:00,1264.333333,8.133333,231.666667
...,...,...,...,...,...
31547136,9664,2022-10-01 18:30:00,70.333333,30.633333,226.033333
31547137,9664,2022-10-01 18:45:00,20.333333,29.966667,226.833333
31547138,9664,2022-10-01 19:00:00,1.333333,29.200000,226.766667
31547139,9664,2022-10-01 19:15:00,0.000000,28.566667,227.000000


In [48]:
pv_timeseries['timestamp'] = pd.to_datetime(pv_timeseries['timestamp'])

In [50]:
start_time = nwp_data['time'][0].values
end_time = nwp_data['time'][-1].values

pv_day_data = pv_timeseries[(pv_timeseries['timestamp'] >= start_time) & (pv_timeseries['timestamp'] <= end_time)]

In [51]:
pv_day_data

,system_id,timestamp,instantaneous_power_W,temperature_C,voltage
119555,10441,2022-01-01 08:30:00,2.500000,NaN,223.500000
119556,10441,2022-01-01 08:45:00,13.333333,NaN,223.000000
119557,10441,2022-01-01 09:00:00,25.333333,NaN,220.666667
119558,10441,2022-01-01 09:15:00,51.666667,NaN,217.666667
119559,10441,2022-01-01 09:30:00,70.333333,NaN,215.000000
...,...,...,...,...,...
31532436,9664,2022-01-01 16:30:00,58.333333,20.000000,227.033333
31532437,9664,2022-01-01 16:45:00,10.000000,19.366667,226.633333
31532438,9664,2022-01-01 17:00:00,0.000000,18.633333,227.266667
31532439,9664,2022-01-01 17:15:00,0.000000,18.066667,226.800000


In [54]:
len(pv_day_data['system_id'].unique())

293

In [55]:
len(pv_data['system_id'].unique())

1325

In [148]:
from tqdm import tqdm

nwp_variable_names = list(nwp_data.variables)

for i, row in tqdm(pv_day_data.iterrows()):
    timestamp = row['timestamp']
    if timestamp.minute != 0:
        continue

    system_id = row['system_id']
    lat = map_val(pv_data[pv_data['system_id'] == system_id]['latitude']).values[0]
    lon = map_val(pv_data[pv_data['system_id'] == system_id]['longitude']).values[0]
    data = nwp_data.sel(time=timestamp, latitude=lat, longitude=lon)
    for var in nwp_variable_names:
        val = data[var].values.item()
        pv_day_data.at[i, var] = val

11800it [19:24, 10.13it/s]


In [149]:
pv_day_data

,system_id,timestamp,instantaneous_power_W,temperature_C,voltage,alnid,alnip,aluvd,aluvp,bld,...,tsrc,ttr,ttrc,u10,u100,u10n,uvb,v10,v100,wind
119555,10441,2022-01-01 08:30:00,2.500000,NaN,223.500000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119556,10441,2022-01-01 08:45:00,13.333333,NaN,223.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119557,10441,2022-01-01 09:00:00,25.333333,NaN,220.666667,0.185514,0.177517,0.042215,0.041346,605.50,...,860905.375,-950268.6875,-923276.6250,-0.204135,0.685549,-0.236828,36905.359375,0.694137,1.303136,NaN
119558,10441,2022-01-01 09:15:00,51.666667,NaN,217.666667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119559,10441,2022-01-01 09:30:00,70.333333,NaN,215.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31532436,9664,2022-01-01 16:30:00,58.333333,20.000000,227.033333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31532437,9664,2022-01-01 16:45:00,10.000000,19.366667,226.633333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31532438,9664,2022-01-01 17:00:00,0.000000,18.633333,227.266667,0.178663,0.169128,0.055082,0.053102,522.75,...,0.000,-943413.5625,-936445.1875,-0.773845,-1.763546,-0.315648,0.000000,-0.718974,0.030087,NaN
31532439,9664,2022-01-01 17:15:00,0.000000,18.066667,226.800000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
pv_day_data.to_csv('processed_pv_data_jan1_2023.csv', index=False)